# HGCAL FE studies

### imports and initial setup

In [1]:
#imports and setup
%matplotlib inline
%cd /home/naodell/work/hgcal/analysis

from time import sleep
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import plotly as py

from descartes import PolygonPatch
from tqdm.notebook import tqdm

#import scripts.helpers as hlp

import matplotlib
matplotlib.rc('font', family='sans-serif') 
matplotlib.rc('font', serif='Helvetica Neue') 
matplotlib.rc('text', usetex='false') 
matplotlib.rcParams.update({'font.size': 16})

#%connect_info

/home/naodell/work/hgcal/analysis


### Load data

In [2]:
path = 'data/pu200'
data_list = []
for i, filename in enumerate(os.listdir(path)):
    infile = open(os.path.join(path, filename), 'rb')
    data_list = pickle.load(infile)
    if i == 10: 
        break

df = pd.concat(data_list)
features = ['zside', 'layer', 'sector', 'panel', 'cell', 'subdet',
            'x', 'y', 'z', 
            'pt', 'reco_e', 'sim_e']


FileNotFoundError: [Errno 2] No such file or directory: 'data/pu200'

### calculate occupancies for panels 

In [ ]:
# sample occupancy plot
layer = 7
df['count'] = np.ones(df.shape[0])
df_avg = df.groupby(['layer', 'panel']).sum()/(2*6*1e4) # sum_hits/(n_sides*n_sectors*n_events)
s_panel = df_avg.query(f'layer == {layer}')['count']
s_panel = s_panel.loc[layer]

df_occ = df_avg.unstack(level=0, fill_value=0)['count'] #pivot(index='panel', layer='layer', values='count')
pd.set_option('precision', 2)

import seaborn as sns
fig, ax = plt.subplots(1, 1, figsize=(12, 10), facecolor='white')
sns.heatmap(df_occ.loc[:19,:25].transpose(), ax=ax, cmap='viridis')
ax.set_title('')

plt.tight_layout()
plt.savefig(f'plots/occupancy.pdf')
plt.show()

In [ ]:
# tmp hack
centers = [( -45.996555,  26.552443), ( -85.601143,  49.418068), (-122.204628,  70.561249),
           ( -52.309528,   8.186293), ( -85.023285,  32.505623), (-126.216042,  56.438793),
           ( -83.052231,  14.980803), (-128.726608,  41.261883), ( -99.199333,   7.929051),
           (-137.761444,  30.080639), ( -48.818104, -11.489166), ( -84.926186, -32.474018),
           (-126.202728, -56.422054), ( -83.127579, -14.974498), (-128.724808, -41.280033),
           (-123.665054, -21.916286), (-132.247528, -10.519128), (-139.554230,  14.877615),
           (-149.350327,  -0.036200)
          ]
patch_file = open(f'data/panel_patches_zplus_{layer}.pkl', 'rb')
patches = pickle.load(patch_file)
patch_file.close()

def invert_ctuple(ctuple):
    return [1 - c for c in ctuple[:-1]] + [1.]

fig, ax = plt.subplots(1, 1, figsize=(12, 12), facecolor='white')

sector = 1
for p, ashape in patches[sector].items():
    if p not in s_panel.index:
        col = plt.cm.viridis(0)
    else:
        col = plt.cm.viridis(s_panel.loc[p]/s_panel.max())
    patch  = PolygonPatch(ashape, fc=col, fill=True, ec=col, linestyle='-', linewidth=3, label='_nolegend_', alpha=1)
    extent = patch.get_extents().bounds
    ax.add_patch(patch)
    
    c = centers[p-1]
    ax.text(c[0]-2, c[1], f'{p}', color=invert_ctuple(col))
    
ax.set_ylim(-80, 90)
ax.set_xlim(-160, -25)
ax.set_xlabel('x [cm]')
ax.set_ylabel('y [cm]')
ax.set_title(f'layer {layer}')
#ax.set_facecolor('white')

dummy = plt.scatter([0, 0], [0, 0], cmap='viridis', c=[0, s_panel.max()])
cbar = plt.colorbar(dummy, ax=ax, label='average occupancy')
cbar.set_alpha(1)
cbar.draw_all()

# draw some concentric circles
df_skim = df.query(f'subdet == 3 and zside == 1 and sector == {sector} and layer == {layer}')
eta   = np.arange(1.6, 3.2, 0.3)
theta = 2*np.arctan(np.exp(-eta))
r     = df_skim.iloc[0].z*np.tan(theta) 
phi   = np.linspace(np.pi/2, 3*np.pi/2, 1000)
xcoord, ycoord  = np.outer(r, np.cos(phi)), np.outer(r, np.sin(phi)) 

for i, (x, y) in enumerate(zip(xcoord, ycoord)):
    ax.plot(x, y, 'r:', label='_nolegend_')
    phi_pos = 7*np.pi/6
    ax.text((r[i] + 10)*np.cos(phi_pos), (r[i] + 10)*np.sin(phi_pos), r'$\eta={0:.1f}$'.format(eta[i]), color='C3', fontsize=10)

plt.grid()
#plt.savefig('plots/panel_numbering_test.png')
plt.savefig('plots/panel_occupancy_test.pdf')
plt.show()

In [ ]:
# sample occupancy plot
g = df.groupby(['layer', 'wafer', 'cell']) # sum_hits/(n_bx*n_sides*n_sectors*n_epochs)
df_avg = g.sum()/(2*6*1e4) # sum_hits/(n_bx*n_sides*n_sectors*n_epochs)
s_cell = df_avg.loc[layer]['count']

cell_map = pd.read_pickle('data/cell_map.pkl')
cell_map = cell_map.loc[1, layer]
#cell_map

In [ ]:
# by cell occupancies
fig, ax = plt.subplots(1, 1, figsize=(12, 12), facecolor='white')

for p, ashape in patches[sector].items():
    patch = PolygonPatch(ashape, fc=col, fill=False, ec='C1', linestyle='-', linewidth=3, label='_nolegend_', alpha=0.7)
    extent = patch.get_extents().bounds
    ax.add_patch(patch)
    
ax.set_ylim(-80, 90)
ax.set_xlim(-160, -25)
ax.set_xlabel('x [cm]')
ax.set_ylabel('y [cm]')
#ax.set_facecolor('white')

df_test = pd.concat([cell_map, s_cell], axis=1).fillna(0)
df_filled = df_test.query('count > 0')
df_empty = df_test.query('count == 0')
ax.scatter(df_empty['tc_x'].values, df_empty['tc_y'].values, c='grey',
           marker='h', 
           s=50, 
           alpha=0.45
           )
scat = ax.scatter(df_filled['tc_x'].values, df_filled['tc_y'].values, c=df_filled['count'].values,
                  cmap='viridis', 
                  marker='h', 
                  vmin=0.,
                  vmax=df_test['count'].max(),
                  s=50, 
                  alpha=0.85
                  )

#dummy = plt.scatter([0, 0], [0, 0], cmap='viridis', c=[0, s_cell.max()])
cbar = plt.colorbar(scat, ax=ax, label='average occupancy')
cbar.set_alpha(1)
cbar.draw_all()

# draw some concentric circles
eta   = np.arange(1.6, 3.2, 0.3)
theta = 2*np.arctan(np.exp(-eta))
r     = df_skim.iloc[0].z*np.tan(theta) 
phi   = np.linspace(np.pi/2, 3*np.pi/2, 1000)
xcoord, ycoord  = np.outer(r, np.cos(phi)), np.outer(r, np.sin(phi)) 

for i, (x, y) in enumerate(zip(xcoord, ycoord)):
    ax.plot(x, y, 'r:', label='_nolegend_')
    phi_pos = 7*np.pi/6
    ax.text((r[i] + 10)*np.cos(phi_pos), (r[i] + 10)*np.sin(phi_pos), r'$\eta={0:.1f}$'.format(eta[i]), color='C3', fontsize=10)

plt.grid()
#plt.savefig('plots/panel_numbering_test.png')
plt.savefig('plots/cell_occupancy_test.pdf')
plt.show()